In [1]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import random
import math
import sys
import json

if './' not in sys.path:
    sys.path.append('./')

In [2]:
getfa = {
"Movie_tr1":{"Directed_by":[0,1,2],"Produced_by":[0,1,2],"Screenplay_by":[1],"Starring":[0,1,2],"Music_by":[0,1,2],"Cinematography":[1]
  ,"Edited_by":[0,1,2],"Productioncompany":[1],"Distributed_by":[0,1,2],"Release_date":[0,1,2],"Running_time":[1]
  ,"Country":[0,1,2],"Language":[0,1,2],"Budget":[1],"Box_office":[1]},
"Book_tr1":{"Publisher":[1],"Schedule":[1],"Format":[0,1,2],"Genre":[0,1,2],"Publication_date":[1],
        "No_of_issues":[1],"Main_character":[0,1,2],"Written_by":[0,1,2]},
"FnD_tr1":{"Manufacturer":[1],"Country_of_origin":[0,1,2],"Variants":[0,1,2],"Introduced":[1],"Related_products":[0,1,2],
    "Alcohol_by_volume":[1],"Website":[1],"Color":[0,1,2],"Main_ingredients":[0,1,2],"Type":[0,1,2]},
"Organiz_tr1":{"Wesbsite":[1],"Headquarters":[1],"Founded":[1],"Industry":[0,1,2],"Key_people":[0,1,2],"Products":[0,1,2]
	,"Number_of_employees":[1],"Traded_as":[0,1,2],"Founder":[0,1,2],"Area_served":[0,1,2],"Type":[1],"Subsidiaries":[0,1,2]
	,"Parent":[1],"Owner":[1],"Predecessor":[1]},
"Paint_tr1":{"Artist":[1],"Year":[1],"Medium":[1],"Dimensions":[1],"Location":[1]},
"Fest_tr1":{"Type":[0,1,2],"Observed_by":[0,1,2],"Frequency":[1],"Celebrations":[0,1,2],"Significance":[0,1,2],"Observances":[0,1,2],
    "Date":[1],"Related_to":[0,1,2],"Also_called":[0,1,2],"Official_name":[1],"Begins":[1],"Ends":[1],
    "2021_date":[1],"2020_date":[1],"2019_date":[1],"2018_date":[1]},
"SpEv_tr1":{"Venue":[0,1,2],"Date":[1],"Competitors":[0,1,2],"Teams":[1],
	"No_of_events":[1],"Established":[1],"Official_site":[1]},
"Univ_tr1":{"Website":[1],"Type":[0,1,2],"Established":[1],"Undergraduates":[1],"Postgraduates":[1],
    "Motto":[0,1,2],"Location":[1],"Nickname":[1],"Campus":[1],"Colors":[0,1,2],
    "Students":[1],"Academic_staff":[1],"Administrative_staff":[1],"President":[1],"Endowment":[1],"Mascot":[1],
    "Provost":[1],"Sporting_affiliations":[0,1,2],"Academic_affiliations":[0,1,2],"Former_names":[1]}
}

In [3]:
Catg = pd.read_csv("/content/drive/My Drive/Auto-TNLI/data/table_categories.tsv",sep="\t")
# Catg = pd.read_csv("../../autotnlidatasetandcode/table_categories modified.tsv",sep="\t")

In [28]:
Ptab = np.array(Catg[Catg.category.isin(['University'])].table_id)
tablesFolder = "/content/drive/My Drive/Auto-TNLI/data/tables"
# tablesFolder = "../../autotnlidatasetandcode/tables"

In [29]:
def parseFile(filename, tablesFolder):
    soup = BeautifulSoup(open(tablesFolder + '/' + filename, encoding="utf8"), 'html.parser')
#     keys =[i.text for i in soup.find('tr').find_all('th')]
    keys = []
#     keys.append(soup.find('caption').text)
    keys =[i.text.replace("\xa0"," ") for i in soup.find('tr').find_all('th')]
    if(soup.find('caption')):
        keys.insert(0,soup.find('caption').text)
    vals = []
    for i in soup.find('tr').find_all('td'):
        if(i.parent.find('th')):
            result = [val.text.strip().replace("\n", "").replace("\t", "") for val in i.find_all('li')]
            if not result:
                if(i.find('br')):
                    for x in i.findAll('br'):
                        x.replace_with(',')
#                 print(i.text)
                    result = i.text.split(',')
                if "â€“" in i.text:
                    result = [val.strip().replace("\n", "").replace("\t", "") for val in i.text.split("â€“")]
                elif " to " in i.text:
                    result = [val.strip().replace("\n", "").replace("\t", "") for val in i.text.split("to")]
                else:
                    result = i.text.strip().replace("\n", "").replace("\t", "")
            vals.append(result)
    title = keys[0]
    dictionary = dict(zip(keys[1:], vals))
    dictionary["Title"] = title
    dictionary["Tablename"] = filename.split(".")[0]
    return dictionary

In [30]:
def get_Table_Title():
    d = {}
    tb = []
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            tb.append(dictionary['Tablename'])
            if("Title" in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary['Title'])
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(dictionary['Title'])
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    return d,tb

In [31]:
N,T = get_Table_Title()
# T

In [32]:
'''
d1 : dict for that table
univ : list of a set
df : dataframe of Born/Death to get the table name
sel: selection bit
it : choose table name from the dataframe
'''
def FakeDICT(tb,dn,univ,di,it,sel=0,subNone = False): # selection bit selects whethet to substitute/delete/add
    d1 = di
    univ = list(univ)
    if(sel==0): # add
        if(d1[tb[it]][0]==None):
            d1[tb[it]]=[]
        ulimit = min(2,len(di[tb[it]])+1) # choose an upper limit of how many to add
        n_add = ulimit
        if(ulimit>1):
            n_add = random.randint(1,ulimit)
        add = random.sample(list(set(univ)-set(d1[tb[it]])),n_add)
        d1[tb[it]] =  list(set(d1[tb[it]]).union(set(add)))
        return d1
    elif(sel==1): 
        if(len(di[tb[it]])>0 and di[tb[it]][0] != None):
            if(len(di[tb[it]])>1):
                keep = random.sample(d1[tb[it]],1)
                ulimit = min(len(list(set(univ)-set(d1[tb[it]]))),len(d1[tb[it]])-1)
                substitute = random.sample(list(set(univ)-set(d1[tb[it]])),ulimit)
            else:
                keep=[]
                substitute = random.sample(list(set(univ)-set(d1[tb[it]])),len(d1[tb[it]]))
            d1[tb[it]] =  list(set(substitute).union(set(keep)))
        elif(len(di[tb[it]])>0):
            possible_sub = random.sample(list(set(univ)-set(d1[tb[it]])),1)
            for i in range(6): # Probability that none is chose = 1/7
                possible_sub.append(random.sample(list(set(univ)-set(d1[tb[it]])),1)[0])
            possible_sub.append(None)
            sub = random.sample(possible_sub,1)
            d1[tb[it]][random.randint(0,len(d1[tb[it]])-1)] = sub[0]
        return d1
    elif(sel==2): # delete nd : for size = 1
        if(len(di[tb[it]])>1 and di[tb[it]][0] != None):
            llimit = max(1,len(d1[tb[it]])-1)
            keep = random.sample(d1[tb[it]], random.randint(1,llimit) ) 
            d1[tb[it]] = keep
        return d1
    
    return None

In [64]:
'''
fake : represent if we have to make a fake dictionary or not
it : selected which index to create the fake for
sel : select whether to add/modify/delete
'''
def get_Website(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    udom = set(["com","in","edu","co","uk"])
    dom = {}
    k = "Website"
    for n in range(37):
        if(int(Ptab[n][1:]) <= 2800):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            d[dictionary['Tablename']] = []
            dom[dictionary["Tablename"]] = []
            if(k in dictionary.keys()):
                u.add(dictionary[k])
                d[dictionary['Tablename']].append(dictionary[k])
#                 print(dictionary[k].split("."))
                if(len(dictionary[k].split("."))>3):
                    for i in range(2,len(dictionary[k].split("."))):
                        dom[dictionary["Tablename"]].append(dictionary[k].split(".")[i])
                else:
                    dom[dictionary["Tablename"]].append(dictionary[k].split(".")[-1])
            else:
                d[dictionary['Tablename']].append(None)
                dom[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(dom[T[it]])<2):
                sel = 1
            dom = FakeDICT(T,N,udom,dom,it,sel)
        
    return list(u),d,dom

In [66]:
# get_Website(T,N)[2]

In [35]:
def get_Type(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Type"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    for i in range(len(dictionary[k])):
                        u.add(dictionary[k][i].strip())
                        d[dictionary['Tablename']].append(dictionary[k][i].strip())
                else:
                    for i in range(len(dictionary[k].split(","))):
                        u.add(dictionary[k].split(",")[i].strip())
                        d[dictionary['Tablename']].append(dictionary[k].split(",")[i].strip())
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [36]:
# getT()[1]

In [37]:
def get_Established(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Established"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
#                     for i in range(len(dictionary[k])):
                    r = " ".join(dictionary[k].replace("\xa0"," "))
                    y = re.findall("[0-9][0-9][0-9][0-9]",r)[0]
                    u.add(y)
                    d[dictionary['Tablename']].append(y)
                else:
#                     for i in range(len(dictionary[k].split(","))):
                    r = dictionary[k].replace("\xa0"," ")
                    y = re.findall("[0-9][0-9][0-9][0-9]",r)[0]
                    u.add(y)
                    d[dictionary['Tablename']].append(y)
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [38]:
# getE()[1]

In [39]:
def get_Undergraduates(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Undergraduates"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
    
                r = dictionary[k]
                n = re.findall("[0-9,]+",r)[0]
                u.add(n.replace(",",""))
                d[dictionary['Tablename']].append(n.replace(",",""))
#                     u.add(dictionary[k])
#                     d[dictionary['Tablename']].append(dictionary[k])
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [40]:
# getUg()[1]

In [41]:
def get_Postgraduates(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Postgraduates"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
    
                r = dictionary[k]
                n = re.findall("[0-9,]+",r)[0]
                u.add(n.replace(",",""))
                d[dictionary['Tablename']].append(n.replace(",",""))
#                     u.add(dictionary[k])
#                     d[dictionary['Tablename']].append(dictionary[k])
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [42]:
# getPg()[1]

In [43]:
def get_Motto(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k1 = "Motto"
    k2 = "Motto in English"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            d[dictionary['Tablename']] = []
            if(k1 in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                if(type(dictionary[k1]) == list):
                    for i in range(len(dictionary[k1])):
                        u.add(dictionary[k1][i])
                        d[dictionary['Tablename']].append(dictionary[k1][i])
                else:
                    u.add(dictionary[k1])
                    d[dictionary['Tablename']].append(dictionary[k1])
            if(k2 in dictionary.keys()):
                if(type(dictionary[k2]) == list):
                    for i in range(len(dictionary[k2])):
                        u.add(dictionary[k2][i])
                        d[dictionary['Tablename']].append(dictionary[k2][i])
                else:
                    u.add(dictionary[k2])
                    d[dictionary['Tablename']].append(dictionary[k2])
                    
            if(k1 not in dictionary.keys() and k2 not in dictionary.keys()):
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"]["Motto"],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [44]:
# getM()[1]

In [76]:
def get_Location(T,N,fake=False,sel=0):
    ul,uc = set([]),set([])
    dl,dc = {},{}
    k = "Location"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                dc[dictionary['Tablename']] = []
                dl[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    r = " ".join(dictionary[k]).replace("\ufeff","")
                    c = re.findall("[A-Za-z]+.[:].+",r)
                    p = re.findall("[A-Za-z,.][A-Za-z,.]+",r)
                    if(not c):
                        p = " ".join(p)
                        dc[dictionary['Tablename']].append(None)
                    else:
                        p = " ".join(p[:-1])[:-1]
                        uc.add(c[0])
                        dc[dictionary['Tablename']].append(c[0])
                    ul.add(p)
                    dl[dictionary['Tablename']].append(p)

                else:
                    r = dictionary[k].replace("\ufeff","")
#                     c = re.findall("[a-z]+.[:].[0-9°′″wnse]+.[0-9°′″wnse]+",r)
                    c = re.findall("[A-Za-z]+.[:].+",r)
                    p = re.findall("[A-Za-z,.][A-Za-z,.]+",r)
                    if(not c):
                        p = " ".join(p)
                        dc[dictionary['Tablename']].append(None)
                    else:
                        p = " ".join(p[:-1])[:-1]
                        uc.add(c[0])
                        dc[dictionary['Tablename']].append(c[0])
                    ul.add(p)
                    dl[dictionary['Tablename']].append(p)
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                dc[dictionary['Tablename']] = []
                dc[dictionary['Tablename']].append(None)
                dl[dictionary['Tablename']] = []
                dl[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(dc[T[it]])<2):
                sel = 1
            dc = FakeDICT(T,N,uc,dc,it,sel)
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(dl[T[it]])<2):
                sel = 1
            dl = FakeDICT(T,N,ul,dl,it,sel)
    
    return list(uc),list(ul),dl,dc

In [77]:
# get_Location(T,N,True)[3]

In [78]:
def get_Nickname(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Nickname"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    for i in range(len(dictionary[k])):
                        u.add(dictionary[k][i])
                        d[dictionary['Tablename']].append(dictionary[k][i])
                else:
                    for i in range(len(dictionary[k].split(","))):
                        u.add(dictionary[k].split(",")[i])
                        d[dictionary['Tablename']].append(dictionary[k].split(",")[i])
#                     u.add(dictionary[k])
#                     d[dictionary['Tablename']].append(dictionary[k])
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [79]:
# getN()[1]

In [80]:
def get_Campus(T,N,fake=False,sel=0):
    u1 = set([])
    u2 = set([])
    d1 = {}
    d2 = {}
    k = "Campus"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d1[dictionary['Tablename']] = []
                d2[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    r = ",".join(dictionary[k]).replace("\xa0"," ")
                    a = re.findall("[0-9][0-9,.]+.[A-Za-z2]+",r)
                    b = re.findall("[A-Za-z]+urban",r)
#                     print(a,b)
                    if(len(a)!=0):
                        for i in range(2):
                            d1[dictionary['Tablename']].append(a[i])
                            u1.add(a[i])
                    if(len(b) != 0):
                        for i in b:
                            d2[dictionary['Tablename']].append(i.lower())
                            u2.add(i.lower())
#                     for i in range(len(dictionary[k])):
#                         u.add(dictionary[k][i].replace("\xa0"," "))
#                         d[dictionary['Tablename']].append(dictionary[k][i].replace("\xa0"," "))
                else:
                    r = dictionary[k].replace("\xa0"," ")
                    a = re.findall("[0-9][0-9,.]+.[A-Za-z2]+",r)
                    b = re.findall("[A-Za-z]+rban",r)
#                     print(a,b)
                    if(len(a)!=0):
                        l = min(len(a),2)
                        for i in range(l):
                            d1[dictionary['Tablename']].append(a[i])
                            u1.add(a[i])
                    if(len(b) != 0):
                        for i in b:
                            d2[dictionary['Tablename']].append(i.lower())
                            u2.add(i.lower())
                if(len(d2[dictionary['Tablename']])<1):
                    d2[dictionary['Tablename']].append(None)
                if(len(d1[dictionary['Tablename']])<1):
                    d1[dictionary['Tablename']].append(None)
#                     for i in range(len(dictionary[k].split(","))):
#                     u.add(dictionary[k].replace("\xa0"," "))
#                     d[dictionary['Tablename']].append(dictionary[k].replace("\xa0"," "))
        
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d1[dictionary['Tablename']] = []
                d1[dictionary['Tablename']].append(None)
                d2[dictionary['Tablename']] = []
                d2[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d1[T[it]])<2):
                sel = 1
            d1 = FakeDICT(T,N,u1,d1,it,sel)
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d2[T[it]])<2):
                sel = 1
            if(len(d2[T[it]])==1):
                d2 = FakeDICT(T,N,u2,d2,it,1)
            elif(len(d2[T[it]])==0):
                d2[T[it]].append(None)
                d2 = FakeDICT(T,N,u2,d2,it,0)
            else:
                d2 = FakeDICT(T,N,u2,d2,it,2)
            
    return list(u1),list(u2),d1,d2

In [81]:
# get_Campus(T,N)[3]

In [82]:
def get_Colors(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Colors"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    for i in range(len(dictionary[k])):
                        u.add(dictionary[k][i].replace(",","").replace("\xa0","").replace("and",",").lower())
                        d[dictionary['Tablename']].append(dictionary[k][i].replace(",","").replace("\xa0","").replace("and",",").lower()) 
                else:
                    
                    r = dictionary[k].replace(",","").replace("\xa0","").replace("and",",").lower()
                    for i in r.split(","):
                        u.add(i.strip())
                        d[dictionary['Tablename']].append(i.strip())

                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [83]:
# getCol()[1]

In [84]:
def get_Students(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Students"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                r = dictionary[k]
                n = re.findall("[0-9,]+",r)[0]
                u.add(n.replace(",",""))
                d[dictionary['Tablename']].append(n.replace(",",""))
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [152]:
# get_Students(T,N)[1][T[26]]

In [86]:
def get_Academic_staff(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Academic staff"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    r = " ".join(dictionary[k])
                    n = re.findall("[0-9,]+",r)[0]
                    u.add(n.replace(",",""))
                    d[dictionary['Tablename']].append(n.replace(",",""))

                else:
                    r = dictionary[k]
                    n = re.findall("[0-9,]+",r)[0]
                    u.add(n.replace(",",""))
                    d[dictionary['Tablename']].append(n.replace(",",""))

                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [87]:
# getAc()[1]

In [88]:
def get_Administrative_staff(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Administrative staff"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    r = " ".join(dictionary[k])
                    n = re.findall("[0-9,]+",r)[0]
                    u.add(n.replace(",",""))
                    d[dictionary['Tablename']].append(n.replace(",",""))

                else:
#                     for i in range(len(dictionary[k].split(","))):
                    r = dictionary[k]
                    n = re.findall("[0-9,]+",r)[0]
                    u.add(n.replace(",",""))
                    d[dictionary['Tablename']].append(n.replace(",",""))

                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [89]:
# getAd()[1]

In [90]:
def get_President(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "President"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    for i in range(len(dictionary[k])):
                        u.add(dictionary[k][i])
                        d[dictionary['Tablename']].append(dictionary[k][i])
                else:
                    for i in range(len(dictionary[k].split(","))):
                        u.add(dictionary[k].split(",")[i])
                        d[dictionary['Tablename']].append(dictionary[k].split(",")[i])
#                     u.add(dictionary[k])
#                     d[dictionary['Tablename']].append(dictionary[k])
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [91]:
# getPr()[1]

In [92]:
def get_Endowment(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Endowment"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                r = dictionary[k]
                m = re.findall("\$[0-9.]+.[a-z]illion",r)
#                 print(m)
                if(not m):
                    d[dictionary['Tablename']].append(dictionary[k])
                else:
                    u.add(m[0])
                    d[dictionary['Tablename']].append(m[0])
#                 u.add(dictionary[k])
#                 d[dictionary['Tablename']].append(dictionary[k])
#                     u.add(dictionary[k])
#                     d[dictionary['Tablename']].append(dictionary[k])
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [93]:
# getEdw()[1]

In [94]:
def get_Mascot(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Mascot"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    for i in range(len(dictionary[k])):
                        u.add(dictionary[k][i])
                        d[dictionary['Tablename']].append(dictionary[k][i])
                else:
#                     for i in range(len(dictionary[k].split(","))):
                    u.add(dictionary[k])
                    d[dictionary['Tablename']].append(dictionary[k])
#                     u.add(dictionary[k])
#                     d[dictionary['Tablename']].append(dictionary[k])
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [95]:
# getMa()[1]

In [96]:
def get_Provost(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Provost"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    for i in range(len(dictionary[k])):
                        u.add(dictionary[k][i])
                        d[dictionary['Tablename']].append(dictionary[k][i])
                else:
                    for i in range(len(dictionary[k].split(","))):
                        u.add(dictionary[k].split(",")[i])
                        d[dictionary['Tablename']].append(dictionary[k].split(",")[i])
#                     u.add(dictionary[k])
#                     d[dictionary['Tablename']].append(dictionary[k])
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [97]:
# getProv()[1]

In [98]:
def get_Sporting_affiliations(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Sporting affiliations"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    for i in range(len(dictionary[k])):
                        u.add(dictionary[k][i])
                        d[dictionary['Tablename']].append(dictionary[k][i])
                else:
#                     for i in range(len(dictionary[k].split(","))):
                    u.add(dictionary[k].replace("\xa0"," "))
                    d[dictionary['Tablename']].append(dictionary[k].replace("\xa0"," "))
#                     u.add(dictionary[k])
#                     d[dictionary['Tablename']].append(dictionary[k])
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [99]:
# getSAf()[1]

In [100]:
def get_Academic_affiliations(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Academic affiliations"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    for i in range(len(dictionary[k])):
                        u.add(dictionary[k][i])
                        d[dictionary['Tablename']].append(dictionary[k][i])
                else:
#                     for i in range(len(dictionary[k].split(","))):
                    u.add(dictionary[k].replace("\xa0"," "))
                    d[dictionary['Tablename']].append(dictionary[k].replace("\xa0"," "))
#                     u.add(dictionary[k])
#                     d[dictionary['Tablename']].append(dictionary[k])
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [101]:
# getAAf()[1]

In [102]:
def get_Former_names(T,N,fake=False,sel=0):
    u = set([])
    d = {}
    k = "Former names"
    for n in range(37):
        if(int(Ptab[n][1:]) <=2800 ):
            dictionary = parseFile(Ptab[n]+".html", tablesFolder)
            if(k in dictionary.keys()):
#                 print(dictionary['Tablename'],' : ',dictionary[k])
                d[dictionary['Tablename']] = []
                if(type(dictionary[k]) == list):
                    for i in range(len(dictionary[k])):
                        u.add(dictionary[k][i].strip())
                        d[dictionary['Tablename']].append(dictionary[k][i].strip())
                else:
                    for i in range(len(dictionary[k].split(","))):
                        u.add(dictionary[k].split(",")[i].replace("\xa0"," ").strip())
                        d[dictionary['Tablename']].append(dictionary[k].split(",")[i].replace("\xa0"," ").strip())
                    
            else:
#                 print(dictionary['Tablename'],':',"!!!")
                d[dictionary['Tablename']] = []
                d[dictionary['Tablename']].append(None)
    if(fake):
        for it in range(37): # for getting all the fakes in one go
            sel = random.sample(getfa["Univ_tr1"][k.replace(" ","_")],1)[0]
            if(sel==2 and len(d[T[it]])<2):
                sel = 1
            d = FakeDICT(T,N,u,d,it,sel)
        
    return list(u),d

In [103]:
# getFn()[1]

#### Extract all data :

In [104]:
def get_Data(fake=False):
    
    Extracted_data = {}
    Keys=["Website","Type","Established","Undergraduates","Postgraduates","Motto","Location"
      ,"Nickname","Campus","Colors","Students","Academic_staff","Administrative_staff","President","Endowment","Mascot"
      ,"Provost","Sporting_affiliations","Academic_affiliations","Former_names"]
    for k in Keys:
        Extracted_data[k]=[]
        for l in eval("get_"+k)(T,N,fake):
            Extracted_data[k].append(l)
            
    return Extracted_data
# F is the Extracted_data[key]

#### Sentences :

In [105]:
def WebsiteSent(tb,dn,F,it,tval=True,prem=False):
    dom = F[2]
    di = F[1]
    univ = F[0]
    syn = [" university"," college"]
    udom = ["in","edu","com","co","uk"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None and dom[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ All+" is this"+random.sample(syn,1)[0]+"'s website",
                    "This"+random.sample(syn,1)[0]+" has a website named "+All,
                    All+" is "+Nm+"'s website"]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None and dom[tb[it]][0] != None):
            length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( "The"+random.sample(syn,1)[0]+" "+Nm+" has the website "+All )
                ts.append( "The website has domain name "+random.sample(dom[tb[it]],1)[0] )
                ts.append( "Information about the faculty or student could be found at "+All )
                ts.append( "You can see the faculty or student profile of university "+Nm+" at "+All )
                ts.append( "Admission info about the college could be found at "+All )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)
                ND = random.sample(list(set(udom)-set(dom[tb[it]])),1)
                All = ','.join(NT)
                ts.append( "The"+random.sample(syn,1)[0]+" "+Nm+" has the website "+All )
                ts.append( "The website has domain name "+random.sample(ND,1)[0] )
                ts.append( "Information about the faculty or student could be found at "+All )
                ts.append( "You can see the faculty or student profile of university "+Nm+" at "+All )
                ts.append( "Admission info about the college could be found at "+All )
                
        else:
            ts.append(None)
            
        return ts

In [106]:
# WebsiteSent(T,N,getW(),1,False)

In [107]:
def TypeSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" start"," initiate"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ All+" are the types of this university"
                  , "The university is of "+All+" type"]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
            length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( Nm+" is a "+",".join(random.sample(di[tb[it]],random.randint(1,length)))+" university" )
                if(re.findall("public",All)):
                    ts.append( Nm+" is a state-funded university" )
                else:
                    ts.append( Nm+" is not a state-funded university" )
                if(re.findall("private",All)):
                    ts.append( Nm+" is a self-funded university" )
                else:
                    ts.append( Nm+" is not a self-funded university" )
                ts.append( Nm+" belongs to "+str(length)+" categories" )
                ts.append( "This university belongs to more than "+str(random.randint(0,length-1))+" categories" )
                ts.append( "This university belongs to less than "+str(random.randint(length+1,length+5))+" categories" )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),random.randint(2,4))
                nlen = len(NT)
                All = ','.join(NT)
                ts.append( Nm+" is a "+",".join(random.sample(NT,random.randint(1,nlen)))+" university" )
                if(re.findall("private",All)):
                    ts.append( Nm+" is a state-funded university" )
                else:
                    ts.append( Nm+" is not a state-funded university" )
                if(re.findall("public",All)):
                    ts.append( Nm+" is a self-funded university" )
                else:
                    ts.append( Nm+" is not a self-funded university" )
                ts.append( Nm+" belongs to "+str(random.randint(length+1,length+5))+" categories" )
                ts.append( "This university belongs to less than "+str(random.randint(1,length))+" categories" )
                ts.append( "This university belongs to more than "+str(random.randint(length,length+5))+" categories" )
                
        else:
            ts.append(None)
            
        return ts

In [108]:
# TSent(T,N,getT()[1],getT()[0],1,False)

In [109]:
def EstablishedSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
    syn = [" founded "," formed "]
    jub = {25:" Silver ",40:" Ruby ",50:" Gold ",60:" Diamond ",65:" Sapphire ",70:" Platinum ",75:" Palladium "
           ,100:" Centennial ",125:" Quasquicentennial ",150:" Sesquicentennial ",175:" Dodransbicentennial ",200:" Bicentennial "}
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ "The university was established in "+All
                  , All+" is when this university was established"
                  , Nm+" was established in "+All ]
        else:
            ps1=[None]
            
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
            year = int(di[tb[it]][0])
            length = len(di[tb[it]])
            if(tval):
#                 All = ','.join(di[tb[it]])
                ts.append( Nm+" was"+random.sample(syn,1)[0]+"in "+str(year) )
                ts.append( Nm+" was"+random.sample(syn,1)[0]+"before "+str(random.randint(year+10,2020)) )
                ts.append( Nm+" was"+random.sample(syn,1)[0]+"after "+str(random.randint(year-100,year-10)) )
                ts.append( Nm+" is "+str(2020-year)+' year old university' )
                ts.append( "The founding stone of the university was laid in "+str(year))
                for i in jub.keys():
                    if(year+i<=2020):
                        ts.append(Nm+" celebrated it's "+jub[i]+"jubilee on "+str(year+i))
                    else:
                        ts.append(Nm+" will celebrate it's "+jub[i]+"jubilee on "+str(year+i))
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)
#                 year = int(NT[0])
                ts.append( Nm+" was"+random.sample(syn,1)[0]+"in "+NT[0] )
                ts.append( Nm+" was"+random.sample(syn,1)[0]+"after "+str(random.randint(year+10,2020)) )
                ts.append( Nm+" was"+random.sample(syn,1)[0]+"before "+str(random.randint(year-100,year-10)) )
                ts.append( Nm+" is "+str(2020-int(NT[0]))+' year old university' )
                ts.append( "The founding stone of the university was laid in "+NT[0])
                year = int(NT[0])
                for i in jub.keys():
                    if(year+i<=2020):
                        ts.append(Nm+" celebrated it's "+jub[i]+"jubilee on "+str(year+i))
                    else:
                        ts.append(Nm+" will celebrate it's "+jub[i]+"jubilee on "+str(year+i))
                
                
        else:
            ts.append(None)
            
        return ts

In [110]:
# ESent(T,N,getE()[1],getE()[0],1,True)

In [111]:
def UndergraduatesSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" university"," college"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ 'There are '+di[tb[it]][0]+' undergraduate students in '+Nm ]
        else:
            ps1=[None]
            
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
            length = len(di[tb[it]])
            num = int(di[tb[it]][0])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( str(num)+" bachelor students studied from this university" )
                ts.append( str(num)+" students are enrolled in the bachelor's program" )
                ts.append( "There are "+str(num)+" undergraduate students on campus every year" )
                ts.append( "There are more than "+str(random.randint(num-80,num-50))+" undergraduate students on campus every year" )
                ts.append( "There are less than "+str(random.randint(num+50,num+100))+" undergraduate students on campus every year" )
                ts.append( Nm+" is expected to graduate "+str(num)+" bachelor students" )
                ts.append( str(num)+" undergraduates study at "+Nm )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)
                ts.append( NT[0]+" bachelor students studied from this university" )
                ts.append( NT[0]+" students are enrolled in the bachelor's program" )
                ts.append( "There are "+NT[0]+" undergraduate students on campus every year" )
                ts.append( "There are less than "+str(random.randint(num-80,num-50))+" undergraduate students on campus every year" )
                ts.append( "There are more than "+str(random.randint(num+50,num+100))+" undergraduate students on campus every year" )
                ts.append( Nm+" is expected to graduate "+NT[0]+" bachelor students" )
                ts.append( NT[0]+" undergraduates study at "+Nm )
                
                
        else:
            ts.append(None)
            
        return ts

In [112]:
# UgSent(T,N,getUg()[1],getUg()[0],1,False)

In [113]:
def PostgraduatesSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" university"," college"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ 'There are '+di[tb[it]][0]+' postgraduate students in '+Nm ]
        else:
            ps1 = [None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
            length = len(di[tb[it]])
            num = int(di[tb[it]][0])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( str(num)+" P.G. students studied from this university" )
                ts.append( str(num)+" students are enrolled in the postgraduate program" )
                ts.append( "There are "+str(num)+" P.G. students on campus every year" )
                ts.append( "There are more than "+str(random.randint(num-80,num-50))+" postgraduate students on campus every year" )
                ts.append( "There are less than "+str(random.randint(num+50,num+100))+" postgraduate students on campus every year" )
                ts.append( Nm+" is expected to graduate "+str(num)+" P.G. students" )
                ts.append( str(num)+" postgraduates study at "+Nm )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)
                ts.append( NT[0]+" P.G. students studied from this university" )
                ts.append( NT[0]+" students are enrolled in the postgraduate program" )
                ts.append( "There are "+NT[0]+" postgraduate students on campus every year" )
                ts.append( "There are less than "+str(random.randint(num-80,num-50))+" postgraduate students on campus every year" )
                ts.append( "There are more than "+str(random.randint(num+50,num+100))+" postgraduate students on campus every year" )
                ts.append( Nm+" is expected to graduate "+NT[0]+" P.G. students" )
                ts.append( NT[0]+" postgraduates study at "+Nm )
                
                
        else:
            ts.append(None)
            
        return ts

In [114]:
# PgSent(T,N,getPg()[1],getPg()[0],1)

In [115]:
def MottoSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" lives by"," believes in"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ All+" is the motto of "+Nm
                  , Nm+" has "+All+" as their motto"]
        else:
            ps1 = [None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( random.sample(di[tb[it]],1)[0]+" are the words the university "+random.sample(["lives by ","believes in "],1)[0] )
                ts.append( random.sample(di[tb[it]],1)[0]+" is the "+random.sample(["slogan ","apopthegm ","platitude "],1)[0]+"of "+Nm )
                ts.append( Nm+random.sample(["stand by ","believe in "],1)[0]+random.sample(di[tb[it]],1)[0] )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),random.randint(1,2))
                ts.append( random.sample(NT,1)[0]+" are the words the university "+random.sample(["lives by ","believes in "],1)[0] )
                ts.append( random.sample(NT,1)[0]+" is the "+random.sample(["slogan ","apopthegm ","platitude "],1)[0]+"of "+Nm )
                ts.append( Nm+random.sample(["stand by ","believe in "],1)[0]+random.sample(NT,1)[0] )
                
        else:
            ts.append(None)
            
        return ts

In [116]:
# MSent(T,N,getM()[1],getM()[0],1)

In [117]:
def LocationSent(tb,dn,F,it,tval=True,prem=False):
    diP = F[2]
    diC = F[3]
    univP = F[1]
    univC = F[0]
    syn = [" at "," located at "," situated at "]
    Nm = dn[tb[it]][0]
    if(prem):
        if(diC[tb[it]][0] != None or diP[tb[it]][0] != None ):
            if(diC[tb[it]][0] != None and diP[tb[it]][0] != None):
                All = ','.join(diC[tb[it]]+diP[tb[it]])
            elif(diC[tb[it]][0] != None):
                All = ','.join(diC[tb[it]])
            else:
                All = ','.join(diP[tb[it]])
            ps1 = [ "The location of this university is "+All
                  , All+" is the location of this university" ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(diP[tb[it]][0] != None or diC[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
#                 All = ','.join(di[tb[it]])
                if(diP[tb[it]][0]!=None):
                    ts.append( Nm+" is"+random.sample(syn,1)[0]+diP[tb[it]][0] )
                    ts.append( Nm+" is"+random.sample(syn,1)[0]+random.sample(diP[tb[it]][0].split(",")[1:],1)[0] )
                if(diC[tb[it]][0]!=None):
                    ts.append( "The coordinates of "+Nm+" is "+diC[tb[it]][0])
            else:
                if(diP[tb[it]][0]!=None):
                    Np = random.sample(list(set(univP)-set(diP[tb[it]])),1)[0]
                    ts.append( Nm+" is"+random.sample(syn,1)[0]+Np )
                    ts.append( Nm+" is"+random.sample(syn,1)[0]+random.sample(Np.split(","),1)[0] )
                if(diC[tb[it]][0]!=None):
                    Nc = random.sample(list(set(univC)-set(diC[tb[it]][0])),1)
                    ts.append( "The coordinates of "+Nm+" is "+Nc[0])
                
        else:
            ts.append(None)
            
        return ts

In [118]:
# LSent(T,N,getL(T,N,0),0,False)

In [119]:
def NicknameSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" lives by"," believes in"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ "The nicknames of the students from this university are "+All
                  , All+" are the nicknames given to the students of "+Nm ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( 'The students of '+Nm+" university are called "+random.sample(di[tb[it]],1)[0] )
                ts.append( random.sample(di[tb[it]],1)[0]+" is graduated from "+Nm )
                ts.append( "Students from "+Nm+" is known as "+random.sample(di[tb[it]],1)[0] )
                ts.append( "Another name for students of "+Nm+" university is "+random.sample(di[tb[it]],1)[0] )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),random.randint(1,2))
                ts.append( 'The students of '+Nm+" university are called "+random.sample(di[tb[it]],1)[0] )
                ts.append( random.sample(di[tb[it]],1)[0]+" is graduated from "+Nm )
                ts.append( "Students from "+Nm+" is known as "+random.sample(di[tb[it]],1)[0] )
                ts.append( "Another name for students of "+Nm+" university is "+random.sample(di[tb[it]],1)[0] )
                
        else:
            ts.append(None)
            
        return ts

In [120]:
# NSent(T,N,getN()[1],getN()[0],2)

In [121]:
def CampusSent(tb,dn,F,it,tval=True,prem=False):
    diU = F[3]
    diA = F[2]
    univU = F[1]
    univA = F[0]
    syn = [" occupies "," take in "]
    Nm = dn[tb[it]][0]
    if(prem):
        if(diA[tb[it]][0] != None):
            All = ','.join(diA[tb[it]])
            if(len(diU[tb[it]])!=0 and diU[tb[it]][0]!=None):
                ps1 = [ "The campus of university is "+All+" ("+diU[tb[it]][0]+")"
                      , All+" ("+diU[tb[it]][0]+")"+" is the campus of this university" ]
            else:
                ps1 = [ "The campus of university is "+All
                      , All+" is the campus of this university" ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if( len(diA[tb[it]])>0 and diA[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
#                 All = ','.join(di[tb[it]])
                ts.append( "The size of "+Nm+" is "+random.sample(diA[tb[it]],1)[0] )
                ts.append( Nm+" is spread in "+random.sample(diA[tb[it]],1)[0] )
                ts.append( Nm+" has an area of "+random.sample(diA[tb[it]],1)[0] )
                ts.append( Nm+random.sample(syn,1)[0]+random.sample(diA[tb[it]],1)[0]+" area" )
                if(len(diU[tb[it]]) != 0 and diU[tb[it]][0] != None ):
                    ts.append( Nm+" is realized in "+diU[tb[it]][0] )
                    ts.append( Nm+" is located in "+diU[tb[it]][0] )
                l = random.sample(diA[tb[it]],1)[0].replace(",","").split(" ")
                n = int(math.ceil(float(l[0])))
                ts.append(Nm+" is less than "+str(random.randint(n+1,n+4))+" "+l[1] )
                ts.append(Nm+" is more than "+str(random.randint(0,n-1))+" "+l[1] )
                
            else:
                NT = random.sample(list(set(univA)-set(diA[tb[it]])),random.randint(1,2))
                
                ts.append( "The size of "+Nm+" is "+random.sample(NT,1)[0] )
                ts.append( Nm+" is spread in "+random.sample(NT,1)[0] )
                ts.append( Nm+" has an area of "+random.sample(NT,1)[0] )
                ts.append( Nm+random.sample(syn,1)[0]+random.sample(NT,1)[0]+" area" )
                if(len(diU[tb[it]]) != 0 and diU[tb[it]][0] != None and len(diU[tb[it]])<2):
                    ts.append( Nm+" is realized in "+random.sample(list(set(univU)-set(diU[tb[it]])),1)[0] )
                    ts.append( Nm+" is located in "+random.sample(list(set(univU)-set(diU[tb[it]])),1)[0] )
                l = random.sample(diA[tb[it]],1)[0].replace(",","").split(" ")
                n = int(math.ceil(float(l[0])))
                ts.append(Nm+" is more than "+str(random.randint(n+1,n+4))+" "+l[1] )
                ts.append(Nm+" is less than "+str(random.randint(0,n-1))+" "+l[1] )
                
        else:
            ts.append(None)
            
        return ts

In [122]:
# CSent(T,N,getC(T,N,0),10,False,True)
# getC(T,N,0)[3]

In [123]:
def ColorsSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" lives by"," believes in"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ "The colors of this university are "+All
                  , All+" are the colors of this university" ]
        else:
            ps1 = [None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( random.sample(di[tb[it]],1)[0]+" is the school color of "+Nm )
                ts.append( random.sample(di[tb[it]],1)[0]+" is one of the school colors of this university" )
                ts.append( random.sample(di[tb[it]],1)[0]+" can be seen at lot of place at "+Nm )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),random.randint(1,2))
                ts.append( random.sample(NT,1)[0]+" is the school color of "+Nm )
                ts.append( random.sample(NT,1)[0]+" is one of the school colors of this university" )
                ts.append( random.sample(NT,1)[0]+" can be seen at lot of place at "+Nm )
                
        else:
            ts.append(None)
            
        return ts

In [124]:
# ColSent(T,N,getCol()[1],getCol()[0],10)

In [125]:
def StudentsSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" university"," college"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ 'There are '+di[tb[it]][0]+' students in '+Nm ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
            length = len(di[tb[it]])
            num = int(di[tb[it]][0])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( str(num)+" students studied from this university" )
                ts.append( str(num)+" students are enrolled in "+Nm )
                ts.append( "There are "+str(num)+" students on campus every year" )
                ts.append( "There are more than "+str(random.randint(num-80,num-50))+" students on campus every year" )
                ts.append( "There are less than "+str(random.randint(num+50,num+100))+" students on campus every year" )
                ts.append( Nm+" is expected to graduate "+str(num)+" students" )
                ts.append( str(num)+" stdents study at "+Nm )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)
                ts.append( NT[0]+" students studied from this university" )
                ts.append( NT[0]+" students are enrolled in "+Nm )
                ts.append( "There are "+NT[0]+" students on campus every year" )
                ts.append( "There are less than "+str(random.randint(num-80,num-50))+" students on campus every year" )
                ts.append( "There are more than "+str(random.randint(num+50,num+100))+" students on campus every year" )
                ts.append( Nm+" is expected to graduate "+NT[0]+" students" )
                ts.append( NT[0]+" students study at "+Nm )
                
                
        else:
            ts.append(None)
            
        return ts

In [126]:
# StSent(T,N,getSt()[1],getSt()[0],1)

In [127]:
def Academic_staffSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" lives by"," believes in"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ All+" academic staff are at "+Nm
                  , Nm+" has "+All+" academic staff" ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( "There are "+All+" full time employees in the university" )
                ts.append( Nm+" employees "+All+" people" )
                ts.append( All+" people  assist in the day to day activities at "+Nm )
                ts.append( All+" people work at "+Nm )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)
                ts.append( "There are "+NT[0]+" full time employees in the university" )
                ts.append( Nm+" employees "+NT[0]+" people" )
                ts.append( NT[0]+" people  assist in the day to day activities at "+Nm )
                ts.append( NT[0]+" people work at "+Nm )
                
        else:
            ts.append(None)
            
        return ts

In [128]:
# AcSent(T,N,getAc()[1],getAc()[0],10,False)

In [129]:
def Administrative_staffSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" lives by"," believes in"]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ All+" administrative staff are at "+Nm
                  , Nm+" has "+All+" administrative staff" ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( "There are "+All+" full time employees in the university" )
                ts.append( Nm+" employees "+All+" people" )
                ts.append( All+" people  assist in the day to day activities at "+Nm )
                ts.append( All+" people work at "+Nm )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)
                ts.append( "There are "+NT[0]+" full time employees in the university" )
                ts.append( Nm+" employees "+NT[0]+" people" )
                ts.append( NT[0]+" people  assist in the day to day activities at "+Nm )
                ts.append( NT[0]+" people work at "+Nm )
                
        else:
            ts.append(None)
            
        return ts

In [130]:
# AdSent(T,N,getAd()[1],getAd()[0],10,False)

In [131]:
def PresidentSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
    syn = [" important "," sensitive "]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ All+" is the president of this university"
                  , "The president of this university is "+All ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( All+" is the head of the institution" )
                ts.append( All+" heads "+Nm+" university" )
                ts.append( All+" is the highest governing body of the university" )
                ts.append( All+" makes the most "+random.sample(syn,1)[0]+" decision of the institution" )
                ts.append( All+" is responsible for the university" )
                ts.append( All+" is the administrator of the university" )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)[0]
                ts.append( NT+" is the head of the institution" )
                ts.append( NT+" heads "+Nm+" university" )
                ts.append( NT+" is the highest governing body of the university" )
                ts.append( NT+" makes the most "+random.sample(syn,1)[0]+" decision of the institution" )
                ts.append( NT+" is responsible for the university" )
                ts.append( NT+" is the administrator of the university" )
                
        else:
            ts.append(None)
            
        return ts

In [132]:
# PrSent(T,N,getPr()[1],getPr()[0],10,False)

In [133]:
def EndowmentSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
    syn = [" received "," was gifted "]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ "The university was endowed "+All
                   ,All+" was endowed to "+Nm ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            m = float(re.findall("[0-9.]+",di[tb[it]][0])[0])
            mm = re.findall("[mb]illion",di[tb[it]][0])[0]
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( "The market financial value of the university is "+All )
                ts.append( "The university was gifted "+All )
                ts.append( "The university "+Nm+random.sample(syn,1)[0]+All+" in donation" )
                ts.append( Nm+random.sample(syn,1)[0]+"more than "+"$"+ str(m-random.randint(1,min(int(m),4)))+" "+mm )
                ts.append( Nm+random.sample(syn,1)[0]+"less than "+"$"+ str(m+random.randint(1,2))+" "+mm )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)[0]
                ts.append( "The market financial value of the university is "+NT )
                ts.append( "The university was gifted "+NT )
                ts.append( "The university "+Nm+random.sample(syn,1)[0]+NT+" in donation" )
                ts.append( Nm+random.sample(syn,1)[0]+"more than "+"$"+ str(m-random.randint(1,min(int(m),4)))+" "+mm )
                ts.append( Nm+random.sample(syn,1)[0]+"less than "+"$"+ str(m+random.randint(1,2))+" "+mm )
                
        else:
            ts.append(None)
            
        return ts

In [134]:
# EdwSent(T,N,getEdw()[1],getEdw()[0],14)

In [135]:
def MascotSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
    syn = [" symbol "," token "]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ "The mascot of this university is "+All
                  , All+" is the mascot of "+Nm]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( All+" represent the "+Nm )
                ts.append( All+" is the"+random.sample(syn,1)[0]+"of "+Nm )
                ts.append( "You can see the symbol "+All+" in every important event of "+Nm)
                ts.append( All+" is the symbol of pride for "+Nm )
                ts.append( "Students are dressed as "+All+" for "+Nm+" games" )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)[0]
                ts.append( NT+" represent the "+Nm )
                ts.append( NT+" is the"+random.sample(syn,1)[0]+"of "+Nm )
                ts.append( "You can see the symbol "+NT+" in every important event of "+Nm)
                ts.append( NT+" is the symbol of pride for "+Nm )
                ts.append( "Students are dressed as "+NT+" for "+Nm+" games" )
                
        else:
            ts.append(None)
            
        return ts

In [136]:
# MaSent(T,N,getMa()[1],getMa()[0],14)

In [137]:
def ProvostSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
    syn = [" important "," sensitive "]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ All+" is the second in head after the president of this university"
                  , "The second in head after the president of this university is "+All ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( All+" is second to the head of the institution" )
#                 ts.append( All+" heads "+Nm+" university" )
                ts.append( All+" is the second highest governing body of the university" )
                ts.append( All+" makes the most "+random.sample(syn,1)[0]+" decision of the institution" )
                ts.append( All+" is responsible for the university" )
                ts.append( All+" is the administrator of the university" )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)[0]
                ts.append( NT+" is second to the head of the institution" )
#                 ts.append( NT+" heads "+Nm+" university" )
                ts.append( NT+" is the second highest governing body of the university" )
                ts.append( NT+" makes the most "+random.sample(syn,1)[0]+" decision of the institution" )
                ts.append( NT+" is responsible for the university" )
                ts.append( NT+" is the administrator of the university" )
                
        else:
            ts.append(None)
            
        return ts

In [138]:
# ProvSent(T,N,getProv()[1],getProv()[0],14)

In [139]:
def Sporting_affiliationsSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
    syn = [" affiliated to "," associated with "," allied with "]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ Nm+" has "+All+" sport affiliations"
                  , "The sport affiliations of "+Nm+" is "+All ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( Nm+" is"+random.sample(syn,1)[0]+All+" in sports" )
                ts.append( "Sports body of the university "+Nm+" is connected to "+All )
                ts.append( All+" regulates the sports of "+Nm )
                
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)[0]
                ts.append( Nm+" is"+random.sample(syn,1)[0]+NT+" in sports" )
                ts.append( "Sports body of the university "+Nm+" is connected to "+NT )
                ts.append( NT+" regulates the sports of "+Nm )
                
        else:
            ts.append(None)
            
        return ts

In [140]:
# SAfSent(T,N,getSAf()[1],getSAf()[0],14)

In [141]:
def Academic_affiliationsSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
    syn = [" affiliated to "," associated with "," allied with "]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ Nm+" has "+All+" academic affiliations"
                  , "The academic affiliations of "+Nm+" is "+All ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
                All = ','.join(di[tb[it]])
                ts.append( Nm+" is"+random.sample(syn,1)[0]+All+" in academics" )
                ts.append( "Academic body of the university "+Nm+" is connected to "+All )
                ts.append( All+" regulates the academics of "+Nm )
                
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),1)[0]
                ts.append( Nm+" is"+random.sample(syn,1)[0]+NT+" in academics" )
                ts.append( "Academic body of the university "+Nm+" is connected to "+NT )
                ts.append( NT+" regulates the academics of "+Nm )
                
        else:
            ts.append(None)
            
        return ts

In [142]:
# AAfSent(T,N,getAAf()[1],getAAf()[0],14)

In [143]:
def Former_namesSent(tb,dn,F,it,tval=True,prem=False):
    di = F[1]
    univ = F[0]
#     syn = [" affiliated to "," associated with "," allied with "]
    Nm = dn[tb[it]][0]
    if(prem):
        if(di[tb[it]][0] != None):
            All = ','.join(di[tb[it]])
            ps1 = [ "The former name of "+Nm+" was "+All
                  , Nm+" was formerly known as "+All ]
        else:
            ps1=[None]
        return ps1
    else:
        ts = []
        if(di[tb[it]][0] != None):
#             length = len(di[tb[it]])
            if(tval):
#                 All = ','.join(di[tb[it]])
                p = random.sample(di[tb[it]],1)[0]
                ts.append( "The past name of "+Nm+" is "+p )
                ts.append( Nm+" was also known by "+p+" in the past" )
                ts.append( Nm+" was previously called as "+p )
                ts.append( p+" changed it's name to "+Nm )
                ts.append( "At some point in time student's used to study at "+p )
                ts.append( Nm+" used to be called "+p+" before" )
                ts.append( p+" was another name for "+Nm+" before" )
                
            else:
                NT = random.sample(list(set(univ)-set(di[tb[it]])),random.randint(1,4))
                p = random.sample(NT,1)[0]
                ts.append( "The past name of "+Nm+" is "+p )
                ts.append( Nm+" was also known by "+p+" in the past" )
                ts.append( Nm+" was previously called as "+p )
                ts.append( p+" changed it's name to "+Nm )
                ts.append( "At some point in time student's used to study at "+p )
                ts.append( Nm+" used to be called "+p+" before" )
                ts.append( p+" was another name for "+Nm+" before" )
                
        else:
            ts.append(None)
            
        return ts

In [144]:
# FnSent(T,N,getFn()[1],getFn()[0],14)

In [158]:
def multi_row1(tb,dn,F,it,tval=True):
    Ug,G = F["Undergraduates"]
    Up,P = F["Postgraduates"]
    Us,S = F["Students"]
    Uc,C = F["Academic_staff"]
    Ud,D = F["Administrative_staff"]
    
    ns = -1 # number of students
    if(G[tb[it]][0]!=None and P[tb[it]][0]!=None):
        ns = int(G[tb[it]][0])+int(P[tb[it]][0])
    elif(S[tb[it]][0]!=None):
        ns = int(S[tb[it]][0])
        
    nst = -1 # number of staff
    if(C[tb[it]][0]!=None and D[tb[it]][0]!=None):
        nst = int(C[tb[it]][0])+int(D[tb[it]][0])
    
    
    Nm = dn[tb[it]][0]
    ts = {}
    if(tval):
        if(ns != -1):
            if(S[tb[it]][0]!=None):
                ts["Students"] = []
                ts["Students"].append( "There are "+str(ns)+" students in "+Nm )
            else:
                ts["Undergraduates,Postgraduates"] = []
                ts["Undergraduates,Postgraduates"].append( "There are "+str(ns)+" students in "+Nm )
        if(nst != -1 and ns != -1):
            if(S[tb[it]][0]!=None):
                ts["Students,Academic_staff,Administrative_staff"] = []
                ts["Students,Academic_staff,Administrative_staff"].append( str(nst)+" employees manage "+str(ns)+" students" )
            else:
                ts["Undergraduates,Postgraduates,Academic_staff,Administrative_staff"] = []
                ts["Undergraduates,Postgraduates,Academic_staff,Administrative_staff"].append( str(nst)+" employees manage "+str(ns)+" students" )
                
    else:
        if(ns != -1):
            if(S[tb[it]][0]!=None):
                ts["Students"] = []
                ts["Students"].append( "There are "+str(random.randint(ns+10,ns+1000))+" students in "+Nm )
            else:
                ts["Undergraduates,Postgraduates"] = []
                ts["Undergraduates,Postgraduates"].append( "There are "+str(random.randint(ns+10,ns+1000))+" students in "+Nm )
                
        if(nst != -1 and ns != -1):
            if(S[tb[it]][0]!=None):
                ts["Students,Academic_staff,Administrative_staff"] = []
                ts["Students,Academic_staff,Administrative_staff"].append( str(random.randint(nst+10,nst+1000))+" employees manage "+str(random.randint(ns+10,ns+1000))+" students" )
            else:
                ts["Undergraduates,Postgraduates,Academic_staff,Administrative_staff"] = []
                ts["Undergraduates,Postgraduates,Academic_staff,Administrative_staff"].append( str(random.randint(nst+10,nst+1000))+" employees manage "+str(random.randint(ns+10,ns+1000))+" students" ) 
        
    return ts

In [160]:
# multi_row1(T,N,get_Data(),26)

In [ ]:
def multi_row2(tb,dn,F,it,tval=True):
    Um,M = F["Mascot"]
    Un,N = F["Nickname"]
    Upr,Pr = F["President"]
    Upv,Pv = F["Provost"]
    
    Nm = dn[tb[it]][0]
    ts = {}
    if(tval):
        if(M[tb[it]][0] != None and N[tb[it]][0] != None):
            ts["Mascot,Nickname"] = []
            Al1 = ",".join(random.sample(M[tb[it]],1))
            Al2 = ",".join(random.sample(N[tb[it]],1))
            ts["Mascot,Nickname"].append( Al1+" is the mascot for the "+Al2 )
        if(Pr[tb[it]][0] != None and Pv[tb[it]][0] != None):
            ts["President,Provost"] = []
            Al1 = ",".join(random.sample(Pr[tb[it]],1))
            Al2 = ",".join(random.sample(Pv[tb[it]],1))
            ts["President,Provost"].append( Al1+" and "+Al2+" are the main administrative figure heads of the university" )
        
        
    else:
        if(M[tb[it]][0] != None and N[tb[it]][0] != None):
            ts["Mascot,Nickname"] = []
            Al1 = ",".join(random.sample(list(set(Um)-set(M[tb[it]])),1))
            Al2 = ",".join(random.sample(list(set(Un)-set(N[tb[it]])),1))
            ts["Mascot,Nickname"].append( Al1+" is the mascot for the "+Al2 )
        if(Pr[tb[it]][0] != None and Pv[tb[it]][0] != None):
            ts["President,Provost"] = []
            Al1 = ",".join(random.sample(list(set(Upr)-set(Pr[tb[it]])),1))
            Al2 = ",".join(random.sample(list(set(Upv)-set(Pv[tb[it]])),1))
            ts["President,Provost"].append( Al1+" and "+Al2+" are the main administrative figure heads of the university" )
        
    return ts

In [147]:
# multi_row2(T,N,0)

In [148]:
def multi_row3(tb,dn,F,it,tval=True):
    Um,M = F["Motto"]
    Us,S = F["Sporting_affiliations"]
    Ua,A = F["Academic_affiliations"]
    
    ME = []
    MOt = []
    if(M[tb[it]][0]!=None):
        for i in M[tb[it]]:
            if(re.findall("latin",i) or re.findall("sankrit",i) or len(re.findall("[A-za-z]",i)) == 0):
                MOt.append(i)
            else:
                ME.append(i)
        
    Nm = dn[tb[it]][0]
    ts = {}
    if(tval):
        if( len(ME) != 0 and len(MOt) != 0 ):
            ts["Motto"] = []
            Al1 = ",".join(random.sample(ME,1))
            Al2 = ",".join(random.sample(MOt,1))
            ts["Motto"].append( "English of the motto "+Al2+" of the university is "+Al1 )
        if(S[tb[it]][0] != None and A[tb[it]][0] != None):
            ts["Sporting_affiliations,Academic_affiliations"] = []
            Al1 = ",".join(random.sample(S[tb[it]],1))
            Al2 = ",".join(random.sample(A[tb[it]],1))
            ts["Sporting_affiliations,Academic_affiliations"].append( "Affiliations of the university are "+Al1+", "+Al2 )
        
    else:
        if( len(ME) != 0 and len(MOt) != 0 ):
            ts["Motto"] = []
            Al1 = ",".join(random.sample(list(set(Um)-set(ME)),1))
            Al2 = ",".join(random.sample(list(set(Um)-set(MOt)),1))
            ts["Motto"].append( "English of the motto "+Al2+" of the university is "+Al1 )
        if(A[tb[it]][0] != None and S[tb[it]][0] != None):
            ts["Sporting_affiliations,Academic_affiliations"] = []
            Al1 = ",".join(random.sample(list(set(Ua)-set(A[tb[it]])),1))
            Al2 = ",".join(random.sample(list(set(Us)-set(S[tb[it]])),1))
            ts["Sporting_affiliations,Academic_affiliations"].append( "Affiliations of the university are "+Al1+", "+Al2 )
        
    return ts

In [146]:
# multi_row3(T,N,get_Data(),13)
# get_Data()["Motto"][1][T[13]]